In [2]:
# script is majorly taken from 
# https://github.com/jontooy/vl_demos/blob/master/feature_extraction_demo.ipynb

In [22]:
import sys
import os
import os.path as op
import yaml
import pandas as pd
import ast
import json
import base64

import numpy as np
np.set_printoptions(suppress=True, precision=4)
sys.path.append("/fsx/harman/sgg_benchmark_pytorch17/scene_graph_benchmark")

In [4]:
winoroot = "/fsx/harman/data/VinVL_img_features/winoground"

In [7]:
# Load height and width of every image
hw_df = pd.read_csv(op.join(winoroot, 'winoground_test.hw.tsv'),sep='\t',header=None,converters={1:ast.literal_eval},index_col=0)

In [47]:
hw_df.iloc[0][1]

[{'height': 1280, 'width': 1920}]

In [17]:
# Directory of out predictions.tsv (bbox_id, class, conf, feature, rect)
sg_tsv = os.path.join(winoroot, 'predictions.tsv')
df = pd.read_csv(sg_tsv,sep='\t',header = None,converters={1:json.loads})#converters={1:ast.literal_eval})
df[1] = df[1].apply(lambda x: x['objects'])

# Help functions
def generate_additional_features(rect,h,w):
    mask = np.array([w,h,w,h],dtype=np.float32)
    rect = np.clip(rect/mask,0,1)
    res = np.hstack((rect,[rect[3]-rect[1], rect[2]-rect[0]]))
    return res.astype(np.float32)

def generate_features(x):
    #image_id, object data list of dictionary, number of detected objects
    idx, data,num_boxes = x[0],x[1],len(x[1])
    # read image height, width, and initialize array of features
    h,w,features_arr = hw_df.loc[idx,1][0]['height'],hw_df.loc[idx,1][0]['width'],[]

    # for every detected object in img
    for i in range(num_boxes):
        # read image region feature vector
        features = np.frombuffer(base64.b64decode(data[i]['feature']),np.float32)
        # add 6 additional dimensions
        pos_feat = generate_additional_features(data[i]['rect'],h,w)
        # stack feature vector with 6 additional dimensions
        x = np.hstack((features,pos_feat))
        features_arr.append(x.astype(np.float32))
        
    features = np.vstack(tuple(features_arr))
    # print(features.shape)
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

def generate_labels(x):
    data = x[1]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

# Generate features from predictions.tsv
df['feature'] = df.apply(generate_features,axis=1)
df['feature'] = df['feature'].apply(json.dumps)

df['label'] = df.apply(generate_labels,axis=1)
df['label'] = df['label'].apply(json.dumps)

In [18]:
df

,0,1,feature,label
0,ex_66_img_1,"[{'rect': [436.1697998046875, 1010.84991455078...","{""features"": ""AAAAAAAAAAAAAAAAAAAAAGNwwz0AAAAA...","[{""class"": ""Short"", ""conf"": 0.919704258441925,..."
1,ex_204_img_1,"[{'rect': [868.9136962890625, 362.998718261718...","{""features"": ""AAAAAKGSAEAAAAAACXKyP3XTY0EE7Ic+...","[{""class"": ""Racket"", ""conf"": 0.974309265613555..."
2,ex_330_img_0,"[{'rect': [571.3433837890625, 689.465942382812...","{""features"": ""qezpPgAAAAAAAAAAl440QGCCN0EjuwJB...","[{""class"": ""Lemon"", ""conf"": 0.9929035305976868..."
3,ex_38_img_1,"[{'rect': [0.0, 280.7784118652344, 1127.890380...","{""features"": ""+rRRPwAAAAAC3Ks/BhEKPwAAAAAAAAAA...","[{""class"": ""Man"", ""conf"": 0.9323487877845764, ..."
4,ex_295_img_0,"[{'rect': [867.616943359375, 740.81396484375, ...","{""features"": ""AAAAAEJukD8AAAAAnFvHP4CXDz4AAAAA...","[{""class"": ""Ice cream"", ""conf"": 0.926971673965..."
...,...,...,...,...
795,ex_322_img_0,"[{'rect': [852.3936767578125, 781.270812988281...","{""features"": ""Eom6PwAAAAAAAAAAAAAAAHCYcz8AAAAA...","[{""class"": ""Whisker"", ""conf"": 0.89565491676330..."
796,ex_248_img_1,"[{'rect': [739.63330078125, 396.25311279296875...","{""features"": ""AAAAAAAAAAAAAAAAVVWAPdfaDz9bCRM+...","[{""class"": ""Glasses"", ""conf"": 0.93224716186523..."
797,ex_151_img_0,"[{'rect': [469.11175537109375, 861.28875732421...","{""features"": ""AAAAAG9NvTxALVA+kn8iQMpv3D8AAAAA...","[{""class"": ""Necklace"", ""conf"": 0.9497052431106..."
798,ex_216_img_1,"[{'rect': [351.962646484375, 242.6018676757812...","{""features"": ""GjYPQIB0Rj8AAAAAb1uWQKcomD4AAAAA...","[{""class"": ""Boat"", ""conf"": 0.9043527841567993,..."


In [ ]:
# Load height and width of every image
hw_df = pd.read_csv(op.join(winoroot, 'winoground_test.hw.tsv'),sep='\t',header=None,converters={1:ast.literal_eval},index_col=0)

# Directory of out predictions.tsv (bbox_id, class, conf, feature, rect)
sg_tsv = os.path.join(winoroot, 'predictions.tsv')
df = pd.read_csv(sg_tsv,sep='\t',header = None,converters={1:json.loads})#converters={1:ast.literal_eval})
df[1] = df[1].apply(lambda x: x['objects'])

In [39]:
# Help functions
def generate_additional_features(rect,h,w):
    mask = np.array([w,h,w,h],dtype=np.float32)
    rect = np.clip(rect/mask,0,1)
    res = np.hstack((rect,[rect[3]-rect[1], rect[2]-rect[0]]))
    return res.astype(np.float32)

def generate_features(x):
    #image_id, object data list of dictionary, number of detected objects
    idx, data, num_boxes = x[0],x[1],len(x[1])

    # read image height, width, and initialize array of features
    h,w,features_arr = hw_df.loc[idx,1][0]['height'],hw_df.loc[idx,1][0]['width'],[]

    # for every detected object in img
    for i in range(num_boxes):
        # read image region feature vector
        features = np.frombuffer(base64.b64decode(data[i]['feature']),np.float32)
        # add 6 additional dimensions
        pos_feat = generate_additional_features(data[i]['rect'],h,w)
        # stack feature vector with 6 additional dimensions
        x = np.hstack((features,pos_feat))
        features_arr.append(x.astype(np.float32))
        
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return features

def generate_num_boxes(x):
    #image_id, object data list of dictionary, number of detected objects
    idx, data, num_boxes = x[0],x[1],len(x[1])

    # read image height, width, and initialize array of features
    h,w,features_arr = hw_df.loc[idx,1][0]['height'],hw_df.loc[idx,1][0]['width'],[]
    return num_boxes

def generate_labels(x):
    data = x[1]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

# Generate features from predictions.tsv
df['feature'] = df.apply(generate_features,axis=1)
# df['feature'] = df['feature'].apply(json.dumps)

df['num_boxes'] = df.apply(generate_num_boxes,axis=1)
# df['num_boxes'] = df['nuum_boxes'].apply(json.dumps)


df['label'] = df.apply(generate_labels,axis=1)
df['label'] = df['label'].apply(json.dumps)

# Generate train/test/val.label.tsv and train/test/val.feature.tsv
TYPE = 'winoground_test'
OUTPUT_DIR = winoroot
LABEL_FILE = os.path.join(OUTPUT_DIR, TYPE+'.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR, TYPE+'.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")
from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
tsv_writer(df[[0,'label']].values.tolist(),LABEL_FILE)
tsv_writer(df[[0, 'num_boxes', 'feature']].values.tolist(),FEATURE_FILE)

In [36]:
df.iloc[0][1]

[{'rect': [436.1697998046875,
   1010.8499145507812,
   805.1593017578125,
   1266.4166259765625],
  'bbox_id': 0,
  'class': 'short',
  'conf': 0.919704258441925,
  'feature': 'AAAAAAAAAAAAAAAAAAAAAGNwwz0AAAAAAAAAAJZpXz0AAAAAECknPwAAAACXjxU/AAAAAAAAAADj0cA+xpadPQAAAAAAAAAASWcAPwAAAADLXA1AAAAAAAAAAAAAAAAASKChPQAAAADGRig8S7jFPgAAAAB9GrM9gPv2PgAAAAAqKyVBAAAAAAAAAAAAAAAAOZRHPk0htT0AAAAA6P0OPyRGSD9KrHg/AAAAAAAAAAAAAAAAAAAAAKBcYD5RCk5A/b5LQAEsMj6mRbc8AAAAAAAAAAAAAAAAAAAAAAAAAADV820/AAAAABRRhT2BE+Y+AAAAADUHAEAAAAAA8/XYPQAAAACLMgc/AAAAAAAAAAAkdEQ/AAAAAAAAAAAAAAAAAAAAADMh6UDqU7c/AAAAAAAAAAAAAAAAAAAAAAAAAACZpIpAAAAAAFdpBz2SuTs/gg/eOwAAAAC5CD4/AAAAALF3jj4AAAAABcI6PhObCT4AAAAAAAAAAAAAAAAKloU9TXxmPgNAAj8EVBc/AAAAAGvSTz2A1yY+jeziPOjgLj7jXklAW5F6QAAAAAAAAAAAhbu7PgAAAAAAAAAAjYRnPQAAAACIo6E/AAAAAAAAAAAAAAAAAAAAACRAKD0AAAAAAAAAAAAAAAAFB6g+AAAAACtaNT4AAAAA9WK3PAAAAAAAAAAAlfy7PgAAAAAAAAAAAAAAAH4rFkDn0cJAAAAAAAAAAAAAAAAAxB21Pb7Gpz8AAAAAAAAAAPcgND8AAAAA03NZPQAAAAAAAAAAXgisP20lKD4AAAAAl+IFPwAAAACd+YE9ddfbPs

In [31]:
df['feature'].iloc[0]

'{"features": "...", "num_boxes": 37}'